In [ ]:
import os
import pandas as pd
import re
import uuid
from sklearn.model_selection import train_test_split

In [ ]:

def preprocess(text):
        input_string =  text   
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        #url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        # url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', java_exp_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        return cleaned_text



In [ ]:
# df = pd.read_csv("../dataset/nlbse-2023-issue-report.csv")
# df.shape
# df = df[df['labels']=='bug']
# print(df.shape)
# df_unique = df.drop_duplicates(subset='id', keep='first')
# print(df_unique.shape)
# df = df_unique
# df["id"] = df["id"].astype(int)
# df_selected = df[['id','body']]

# concatenated_df = pd.read_csv("../dataset/Manual-labelled-data-25K.csv")
# concatenated_df_selected = concatenated_df[['Issue_id','Candidate_String', 'is_secret_human_label']]

# # Rename columns for consistency
# df_selected = df_selected.rename(columns={'body': 'text'})
# concatenated_df_selected = concatenated_df_selected.rename(columns={'Candidate_String': 'candidate_string', 'is_secret_human_label': 'label'})

# # Merge the two DataFrames on a common key, for example, 'id'
# # Adjust the merge key as needed based on your data
# merged_df = concatenated_df_selected.merge(df_selected, left_on='Issue_id', right_on='id')
# print(merged_df.shape)
# columns_to_remove = ['id']
# merged_df.drop(columns=columns_to_remove, inplace=True)
# print(merged_df.columns)



In [ ]:
import pandas as pd

def build_preprocessed_df(df, text_col="text", substr_col="candidate_string"):
    """
    Keep rows where substr_col is a substring of text_col.
    Also return counts of matches and mismatches.

    Args:
        df (pd.DataFrame): Input dataframe
        text_col (str): Column with the main string
        substr_col (str): Column with candidate substrings

    Returns:
        preprocessed_df (pd.DataFrame): Rows where substring is found in text
        stats (dict): {"Ok": match_count, "Mismatch": mismatch_count}
    """
    preprocessed_df = None
    count = 0
    inverse_count = 0

    for i in range(df.shape[0]):
        main_string = df[text_col].iloc[i]
        substring = df[substr_col].iloc[i]

        if substring not in main_string:
            count += 1
        else:
            inverse_count += 1
            # Append row to preprocessed_df
            row_df = pd.DataFrame(df.iloc[i, :]).transpose()
            if preprocessed_df is None:
                preprocessed_df = row_df
            else:
                preprocessed_df = pd.concat([preprocessed_df, row_df], ignore_index=True)

    if preprocessed_df is None:
        preprocessed_df = pd.DataFrame(columns=df.columns)  # empty case

    print(f"Mismatch: {count}")
    print(f"Ok: {inverse_count}")
    print(preprocessed_df.shape)

    return preprocessed_df


In [ ]:
def check_substrings(df, text_col="text", substr_col="candidate_string"):
    """
    Check if each substring in `substr_col` exists in the corresponding row of `text_col`.
    
    Args:
        df (pd.DataFrame): Input dataframe.
        text_col (str): Name of the column containing main text.
        substr_col (str): Name of the column containing candidate substring.
    
    Returns:
        dict: A dictionary with counts of matches and mismatches.
    """
    count = 0
    inverse_count = 0
    
    for i in range(df.shape[0]):
        main_string = df[text_col].iloc[i]
        substring = df[substr_col].iloc[i]
        if substring in main_string:
            count += 1
        else:
            inverse_count += 1
    
    print(f"Mismatch: {inverse_count}")
    print(f"Ok: {count}")
    
    return {"ok": count, "mismatch": inverse_count}


In [ ]:
# check_substrings(merged_df)
# # merged_df["text"] = merged_df["text"].apply(preprocess)


In [ ]:
# count=0
# inverse_count=0
# #print(preprocessed_df['modified_text'][2])
# for i in range(merged_df.shape[0]):
#   #print(i)
#   main_string=merged_df['text'][i]
#   substring=merged_df['candidate_string'][i]
#   if substring in main_string:
#     count+=1
#   else:
#     inverse_count+=1
#     merged_df['text'][i] = preprocess(merged_df["text"][i])
# print("Mismatch: "+str(inverse_count))
# print("Ok: "+str(count))

In [ ]:
# check_substrings(merged_df)
# preprocessed_df = build_preprocessed_df(merged_df)
# preprocessed_df.to_csv("dataset/og_all.csv")

In [ ]:
import pandas as pd

def load_and_count_labels(file_path: str, label_column: str = 'label'):
    """
    Load a dataframe from a CSV file and count the number of 0s and 1s in the label field.

    Args:
        file_path (str): Path to the CSV file.
        label_column (str): Name of the label column. Defaults to 'label'.

    Returns:
        dict: A dictionary with counts of 0s and 1s.
    """
    # Load the DataFrame
    df = pd.read_csv(file_path)
    
    # Ensure label column exists
    if label_column not in df.columns:
        raise ValueError(f"Column '{label_column}' not found in the dataframe.")
    
    # Count 0s and 1s
    counts = df[label_column].value_counts().to_dict()
    
    # Report results
    zero_count = counts.get(0, 0)
    one_count = counts.get(1, 0)
    print(f"Label counts - 0s: {zero_count}, 1s: {one_count}")
    
    return {'0': zero_count, '1': one_count}


In [ ]:
# df_0 = preprocessed_df[preprocessed_df["label"] == 0].reset_index(drop=True)
# df_1 = preprocessed_df[preprocessed_df["label"] == 1].reset_index(drop=True)
# df_1.to_csv("dataset/og_pos.csv")
# df_0.to_csv("dataset/og_neg.csv")


In [ ]:
load_and_count_labels("dataset/og_pos.csv")
load_and_count_labels("dataset/og_neg.csv")

In [ ]:
import uuid
df1 = pd.read_csv("dataset/compiled_tp.csv")
df2 = pd.read_csv("dataset/compiled_tp_2.csv")
df3 = pd.read_csv("dataset/compiled_tp_3.csv")
df4 = pd.read_csv("dataset/compiled_tp_4.csv")
df5 = pd.read_csv("dataset/compiled_tp_5.csv")
df_combined = pd.concat([df1, df2,df3,df4,df5], ignore_index=True)
df_combined = df_combined[['candidate_string','body']]
df_combined = df_combined.rename(columns={'body': 'text'})
df_combined["Issue_id"] = [uuid.uuid4() for _ in range(len(df_combined))]
df_combined["label"] = 1
df_combined.head()
df_combined["candidate_string"] = df_combined["candidate_string"].astype(str) 
df_combined["text"] = df_combined["text"].astype(str) 


In [ ]:
check_substrings(df_combined)
count=0
inverse_count=0
#print(preprocessed_df['modified_text'][2])
for i in range(df_combined.shape[0]):
  #print(i)
  main_string=df_combined['text'][i]
  substring=df_combined['candidate_string'][i]
  if substring in main_string:
    count+=1
  else:
    inverse_count+=1
    df_combined['text'][i] = preprocess(df_combined["text"][i])
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))
check_substrings(df_combined)
preprocessed_df = build_preprocessed_df(df_combined)


In [ ]:
preprocessed_df.to_csv("dataset/new_pos.csv")
#preprocessed_df.head()

In [ ]:
import uuid
df1 = pd.read_csv("dataset/compiled_fp.csv")
df2 = pd.read_csv("dataset/compiled_fp_2.csv")
df3 = pd.read_csv("dataset/compiled_fp_3.csv")
df_combined = pd.concat([df1, df2, df3], ignore_index=True)
df_combined = df_combined[['candidate_string','body']]
df_combined = df_combined.rename(columns={'body': 'text'})
df_combined["Issue_id"] = [uuid.uuid4() for _ in range(len(df_combined))]
df_combined["label"] = 0
df_combined.head()
df_combined["candidate_string"] = df_combined["candidate_string"].astype(str) 
df_combined["text"] = df_combined["text"].astype(str) 

In [ ]:
check_substrings(df_combined)
count=0
inverse_count=0
#print(preprocessed_df['modified_text'][2])
for i in range(df_combined.shape[0]):
  #print(i)
  main_string=df_combined['text'][i]
  substring=df_combined['candidate_string'][i]
  if substring in main_string:
    count+=1
  else:
    inverse_count+=1
    df_combined['text'][i] = preprocess(df_combined["text"][i])
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))
check_substrings(df_combined)
preprocessed_df = build_preprocessed_df(df_combined)

In [ ]:
preprocessed_df.to_csv("dataset/new_neg.csv")
preprocessed_df.head()

In [ ]:
load_and_count_labels("dataset/og_neg.csv")
load_and_count_labels("dataset/og_pos.csv")
load_and_count_labels("dataset/new_neg.csv")
load_and_count_labels("dataset/new_pos.csv")

In [ ]:
df1 = pd.read_csv("dataset/og_neg.csv")
df2 = pd.read_csv("dataset/new_neg.csv")
#df_neg = pd.concat([df1, df2], ignore_index=True)
df_neg = df2

df1 = pd.read_csv("dataset/og_pos.csv")
df2 = pd.read_csv("dataset/new_pos.csv")
#df_pos = pd.concat([df1, df2], ignore_index=True)
df_pos = df2

In [ ]:
# df_neg1,df_neg2= train_test_split(
#     df_neg,
#     test_size=0.5,
#     random_state=42,
# )

In [ ]:
pos_train_val_df,pos_test_df= train_test_split(
    df_pos,
    test_size=0.15,
    random_state=42,
)
pos_train_df,pos_val_df= train_test_split(
    pos_train_val_df,
    test_size=0.118,
    random_state=42,
)

neg_train_val_df,neg_test_df= train_test_split(
    df_neg,
    test_size=0.15,
    random_state=42,
)
neg_train_df,neg_val_df= train_test_split(
    neg_train_val_df,
    test_size=0.118,
    random_state=42,
)

In [ ]:
df_train = pd.concat([pos_train_df, neg_train_df], ignore_index=True)
df_val = pd.concat([pos_val_df, neg_val_df], ignore_index=True)
df_test = pd.concat([pos_test_df, neg_test_df], ignore_index=True)

In [ ]:
df_train["Issue_id"] = [uuid.uuid4() for _ in range(len(df_train))]
df_val["Issue_id"] = [uuid.uuid4() for _ in range(len(df_val))]
df_test["Issue_id"] = [uuid.uuid4() for _ in range(len(df_test))]

In [ ]:
import pandas as pd
import numpy as np

def process_dataframe(input_df: pd.DataFrame):
    """
    Prepare X_text, X_candidate, Y_labels from a raw dataframe.
    - Cleans NaN/±inf
    - Casts text/candidate to string
    - Casts label to int (dropping NaN labels)
    - Builds clean_text and modified_text columns
    Returns: (X_text, X_candidate, Y_labels)
    """

    # --- 1) Validate input ---
    if not isinstance(input_df, pd.DataFrame):
        raise TypeError("Input must be a Pandas DataFrame.")
    required_cols = {"Issue_id","text", "candidate_string", "label"}
    missing = required_cols - set(input_df.columns)
    if missing:
        raise KeyError(f"Missing required column(s): {missing}")

    if input_df.empty:
        print("Warning: Input DataFrame is empty.")
        return [], [], []

    # --- 2) Work on a copy ---
    df = input_df.copy()

    # --- 3) Clean candidate_string & text: replace inf, drop NaN if you need strict non-empty ---
    for col in ["candidate_string", "text"]:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        # If you prefer dropping rows where these are missing, uncomment next line:
        df = df.dropna(subset=[col])
        # Otherwise, keep rows but fill with empty string to avoid tokenizer errors:
        #df[col] = df[col].astype("string").fillna("")

    # --- 4) Clean label: replace inf→NaN, drop NaN, cast to int ---
    df["label"] = df["label"].replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=["label"])
    df["label"] = df["label"].astype(int)

    return df


In [ ]:
df_train= process_dataframe(df_train)
df_val= process_dataframe(df_val)
df_test= process_dataframe(df_test)


In [ ]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
df_val = df_val.sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df_test.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_train.to_csv("dataset/train.csv")
df_val.to_csv("dataset/val.csv")
df_test.to_csv("dataset/test.csv")

In [ ]:
load_and_count_labels("dataset/train.csv")
load_and_count_labels("dataset/val.csv")
load_and_count_labels("dataset/test.csv")